# League of Legends Predictor

**Name(s)**: Jawad Najar and Ali Boussi

**Website Link**: (your website link)

In [180]:
import pandas as pd
import numpy as np

import plotly.express as px
pd.options.plotting.backend = 'plotly'
from lec_utils import * # Feel free to uncomment and use this. It'll make your plotly graphs look like ours in lecture!
pd.set_option('display.max_columns', None)

## Step 1: Introduction

In [181]:
df = pd.read_csv('data.csv')

In [182]:
df.columns[df.columns.str.contains('xp')]
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,participantid,side,position,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,void_grubs,opp_void_grubs,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,gpr,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,goldat20,xpat20,csat20,opp_goldat20,opp_xpat20,opp_csat20,golddiffat20,xpdiffat20,csdiffat20,killsat20,assistsat20,deathsat20,opp_killsat20,opp_assistsat20,opp_deathsat20,goldat25,xpat25,csat25,opp_goldat25,opp_xpat25,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,1,Blue,top,Soboro,oe:player:38e0af7278d6769d0c81d7c4b47ac1e,BRION Challengers,oe:team:733ebb9dbf22a401c0127a0c80193ca,Renekton,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,1713,0,2,3,2,9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.32,0.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,15768.0,552.29,0.28,1072.40,777.79,8.0,0.28,6.0,0.21,5.0,26.0,0.91,10934,7164.0,250.93,0.25,10275.0,NaN,NaN,231.0,220.0,11.0,NaN,NaN,8.09,3228.0,4909.0,89.0,3176.0,4953.0,81.0,52.0,-44.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,5025.0,7560.0,135.0,4634.0,7215.0,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0,6506.0,9853.0,172.0,6338.0,10200.0,171.0,168.0,-347.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,8462.0,11754.0,212.0,7857.0,12279.0,203.0,605.0,-525.0,9.0,0.0,1.0,1.0,0.0,2.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,2,Blue,jng,Raptor,oe:player:637ed20b1e41be1c51bd1a4cb211357,BRION Challengers,oe:team:733ebb9dbf22a401c0127a0c80193ca,Xin Zhao,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,1713,0,2,5,6,9,19,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.32,0.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,11765.0,412.08,0.21,944.27,650.16,6.0,0.21,18.0,0.63,6.0,48.0,1.68,9138,5368.0,188.02,0.19,8750.0,NaN,NaN,148.0,33.0,115.0,NaN,NaN,5.18,3429.0,3484.0,58.0,2944.0,3052.0,63.0,485.0,432.0,-5.0,1.0,2.0,0.0,0.0,0.0,1.0,5366.0,5320.0,89.0,4825.0,5595.0,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0,6854.0,7193.0,116.0,6708.0,8275.0,142.0,146.0,-1082.0,-26.0,2.0,3.0,2.0,1.0,5.0,1.0,8254.0,8958.0,135.0,7833.0,9861.0,163.0,421.0,-903.0,-28.0,2.0,4.0,2.0,1.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,3,Blue,mid,Feisty,oe:player:d1ae0e2f9f3ac1e0e0cdcb86504ca77,BRION Challengers,oe:team:733ebb9dbf22a401c0127a0c80193ca,LeBlanc,Karma,Caitlyn,Syndra,Thresh,Lulu,NaN,NaN,NaN,NaN,NaN,1713,0,2,2,3,9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.32,0.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0

## Step 2: Data Cleaning and Exploratory Data Analysis

In [ ]:
gvis_diff = df.copy()
gvis_diff = gvis_diff[(df['datacompleteness'] == 'complete') & (df['position'] != 'team')]
gvis_diff = gvis_diff[['position', 'playername', 'earnedgold', 'total cs']]

gvis_diff['earnedgold'] = (gvis_diff['earnedgold'] - gvis_diff['earnedgold'].mean()) / gvis_diff['earnedgold'].std()
gvis_diff['total cs'] = (gvis_diff['total cs'] - gvis_diff['total cs'].mean()) / gvis_diff['total cs'].std()

In [249]:
gold_stats = df.copy()
gold_stats = gold_stats[(gold_stats['datacompleteness'] == 'complete') & (gold_stats['position'] == 'team')][['gameid', 'earnedgold', 'side', 'result']]
gold_stats['red_win'] = ((gold_stats['result'] == 1) & ((gold_stats['side'] == 'Red')))
red_side_gold = gold_stats.loc[gold_stats['side'] == 'Red', ['gameid', 'earnedgold', 'red_win']]
blue_side_gold =  gold_stats.loc[gold_stats['side'] == 'Blue', ['gameid', 'earnedgold']]
gold_final = red_side_gold.merge(blue_side_gold, on='gameid', suffixes=('_red', '_blue'))

fig = px.scatter(
    gold_final,
    x='earnedgold_red',
    y='earnedgold_blue',
    color='red_win',
    color_discrete_map={True: '#D2042D', False: '#0047AB'},
    opacity=0.6,
    hover_data={
        'earnedgold_red': ':.0f',
        'earnedgold_blue': ':.0f',
        'red_win': True
    },
    labels={
        'earnedgold_red': 'Red Team Gold',
        'earnedgold_blue': 'Blue Team Gold',
        'red_win': 'Red Team Won'
    },
    title='Red Gold Earned vs Blue Gold Earned by Game Outcome'
)

fig.update_traces(marker=dict(size=6))

fig.update_layout(
    width=900,
    height=600,
    title_font_size=20,
    xaxis=dict(title='Red Team Earned Gold', tickformat=',', title_font_size=16),
    yaxis=dict(title='Blue Team Earned Gold', tickformat=',', title_font_size=16),
    legend_title_text='Red Team Win?',
    legend=dict(font=dict(size=12))
)

fig.show()

## Step 3: Framing a Prediction Problem

In [ ]:
# TODO

## Step 4: Baseline Model

In [ ]:
# TODO

## Step 5: Final Model

In [ ]:
# TODO